In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime
import asyncio  # 이 줄을 추가
import urllib3
import ssl
import requests
import os
from pathlib import Path
import sys
print(f"Python version: {sys.version}")
print(f"Requests version: {requests.__version__}")
# 브라우저 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # 헤드리스 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')  # Linux에서 필요
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('--remote-debugging-port=9222')  # 디버깅 포트 추가
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36')

# 추가 설정
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
chrome_options.add_experimental_option('useAutomationExtension', False)

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Requests version: 2.32.3


In [2]:
def get_pdf_download_url(file_name):
    if not file_name:
        return ""
    base_url = "https://www.hanwhalife.com/main/disclosure/goods/download_chk.asp"
    return f"{base_url}?file_name={file_name}"

import warnings
import urllib3
warnings.filterwarnings('ignore', category=urllib3.exceptions.InsecureRequestWarning)

def check_file_pattern(file_name):
    """
    파일명 패턴 체크
    """
    import re
    pattern = r'^[a-zA-Z0-9_-]'
    return bool(re.match(pattern, file_name))

def get_pdf_download_url(file_name, is_stopped=False):
    """
    PDF 다운로드 URL 생성
    Args:
        file_name (str): 파일명
        is_stopped (bool): 판매중지 상품 여부
    Returns:
        str: 다운로드 URL
    """
    if not file_name:
        return ""
        
    # 판매중지 상품은 다른 URL 사용
    base_url = "https://www.hanwhalife.com/www/announce/goods/"
    download_path = "download_chk_stop.asp" if is_stopped else "download_chk.asp"
    return f"{base_url}{download_path}?file_name={file_name}"

def download_pdf(file_name, is_stopped=False):
    """
    PDF 파일을 다운로드하는 함수
    """
    if not file_name:
        return ""
        
    try:
        Path("./data/pdf_docs").mkdir(parents=True, exist_ok=True)
        file_path = os.path.join("./data/pdf_docs", file_name)
        
        if os.path.exists(file_path):
            print(f"이미 존재하는 파일: {file_name}")
            return file_name
            
        # 세션 생성
        session = requests.Session()
        session.verify = False
        
        # 파일명 패턴 체크하여 적절한 URL 선택
        if is_stopped and check_file_pattern(file_name):
            download_url = "https://file.hanwhalife.com/www/announce/goods/download_chk_stop.asp"
        else:
            download_url = "https://file.hanwhalife.com/www/announce/goods/download_chk.asp"
        
        headers = {
            'accept': '*/*',
            'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'x-requested-with': 'XMLHttpRequest',
            'referer': 'https://www.hanwhalife.com/'
        }
        
        # 파일명 EUC-KR 인코딩
        encoded_filename = file_name.encode('euc-kr')
        
        data = {
            'file_name': encoded_filename
        }
        
        response = session.post(
            download_url,
            data=data,
            headers=headers,
            allow_redirects=True,
            timeout=30
        )
        
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"다운로드 완료: {file_name}")
            return file_name
        else:
            print(f"다운로드 실패: {file_name}")
            return ""
            
    except Exception as e:
        print(f"파일 다운로드 중 오류 발생: {str(e)}")
        return ""
    
async def get_product_details(driver, sell_type, goods_type):
    try:
        # 상품 목록 요청을 위한 JavaScript 코드
        js_code = """
            var done = arguments[arguments.length - 1];
            
            fetch('/main/disclosure/goods/goodslist/getList2.do', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/x-www-form-urlencoded',
                    'X-Requested-With': 'XMLHttpRequest',
                    'Accept': 'application/json, text/javascript, */*; q=0.01'
                },
                body: new URLSearchParams({
                    'PType': '1',
                    'sellFlag': 'Y',
                    'goodsType': arguments[0],
                    'sellType': arguments[1],
                    'goodsIndex': '',
                    'schText': ''
                })
            })
            .then(response => response.json())
            .then(data => done(data))
            .catch(error => done({error: error.message}));
        """
        
        # 요청 실행 (첫 번째 요청과 동일한 방식 사용)
        data = driver.execute_async_script(js_code, goods_type, sell_type)
        
        if data and 'list2' in data:
            print(f"상품 목록 조회 성공: {len(data['list2'])} 개 상품")
            return data
        else:
            print("상품 목록 없음")
            return None
            
    except Exception as e:
        print(f"데이터 요청 중 오류 발생: {str(e)}")
        return None

# 브라우저 설정도 추가로 수정
chrome_options.add_argument('--disable-web-security')  # CORS 제한 해제
chrome_options.add_argument('--allow-running-insecure-content')  # 혼합 콘텐츠 허용
chrome_options.add_argument('--disable-features=IsolateOrigins,site-per-process')  # 프로세스 격리 해제

async def get_product_specific_info(driver, idx):
    try:
        # 상품 상세 정보 요청을 위한 JavaScript 코드
        js_code = """
            var done = arguments[arguments.length - 1];
            
            fetch('/main/disclosure/goods/goodslist/getList3.do', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/x-www-form-urlencoded',
                    'X-Requested-With': 'XMLHttpRequest',
                    'Accept': 'application/json, text/javascript, */*; q=0.01'
                },
                body: new URLSearchParams({
                    'PType': '1',
                    'sellFlag': 'Y',
                    'goodsIndex': arguments[0]
                })
            })
            .then(response => response.json())
            .then(data => done(data))
            .catch(error => done({error: error.message}));
        """
        
        result = driver.execute_async_script(js_code, idx)
        return result
        
    except Exception as e:
        print(f"상세 정보 요청 중 오류: {str(e)}")
        return None

async def process_url(driver, url, sale_status):
    """
    지정된 URL의 상품 정보를 처리하는 함수
    """
    driver.get(url)
    print(f"\n{sale_status} 상품 페이지 접속 완료")
    
    # JavaScript 로딩 대기
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.ID, "LIST_GRID1")))
    time.sleep(3)
    
    # fetch API를 사용한 데이터 요청
    js_code = """
        var done = arguments[arguments.length - 1];
        
        fetch('/main/disclosure/goods/goodslist/getList.do', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
                'X-Requested-With': 'XMLHttpRequest',
                'Accept': '*/*'
            },
            body: new URLSearchParams({
                'PType': '1',
                'sellFlag': arguments[0],  // 판매상태에 따라 'Y' 또는 'N' 설정
                'schText': ''
            })
        })
        .then(response => response.json())
        .then(data => done(data))
        .catch(error => done({error: error.message}));
    """
    
    print("데이터 요청 시작")
    sell_flag = 'Y' if sale_status == "판매중" else 'N'
    initial_data = driver.execute_async_script(js_code, sell_flag)
    print("응답 데이터:", initial_data)
    
    products_data = []
    
    if isinstance(initial_data, dict) and not 'error' in initial_data:
        if 'list1' in initial_data:
            for item in initial_data['list1']:
                print(f"\n처리 중: {item['SELL_TYPE_NM']} {item['GOODS_TYPE_NM']}")
                result = await get_product_details(driver, item['SELL_TYPE'], item['GOODS_TYPE'])
                
                if result and 'list2' in result:
                    for product in result['list2']:
                        print(f"상품 상세 정보 수집 중: {product['GOODS_NAME']}")
                        detail = await get_product_specific_info(driver, product['IDX'])
                        
                        if detail and 'list3' in detail:
                            for doc in detail['list3']:
                                # PDF 파일 다운로드 및 파일명 저장
                                is_stopped = (sale_status == "판매중지")
                                summary_file = download_pdf(doc.get('FILE_NAME1', ''), is_stopped)
                                method_file = download_pdf(doc.get('FILE_NAME2', ''), is_stopped)
                                terms_file = download_pdf(doc.get('FILE_NAME3', ''), is_stopped)
                                
                                product_info = {
                                    '판매구분': sale_status,
                                    '판매사': '한화생명',
                                    '분류': f"{item['SELL_TYPE_NM']} {item['GOODS_TYPE_NM']}",
                                    '상품명': product['GOODS_NAME'],
                                    '판매기간': f"{doc.get('SELL_START_DT', '')} ~ {doc.get('SELL_END_DT', '')}",
                                    '요약서': summary_file,
                                    '방법서': method_file,
                                    '약관': terms_file
                                }
                                products_data.append(product_info)
                                
                                await asyncio.sleep(1)
                        
                        await asyncio.sleep(1)
    
    return products_data

try:
    # 브라우저 시작
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # URL 설정
    url1 = 'https://www.hanwhalife.com/main/disclosure/goods/goodslist/DF_GDGL000_P10000.do'
    url2 = 'https://www.hanwhalife.com/main/disclosure/goods/goodslist/DF_GDGL000_P20000.do'
    
    # 판매중 상품 처리
    # selling_products = await process_url(driver, url1, "판매중")
    # print(f"\n판매중 상품 {len(selling_products)}개 처리 완료")
    
    # 판매중지 상품 처리
    stopped_products = await process_url(driver, url2, "판매중지")
    print(f"\n판매중지 상품 {len(stopped_products)}개 처리 완료")
    
    # 전체 상품 데이터 통합
    # all_products = selling_products + stopped_products
    all_products = stopped_products
    
    # DataFrame 생성 및 Excel 파일 저장
    df = pd.DataFrame(all_products)
    
    # 컬럼 순서 지정
    columns = ['판매구분', '판매사', '분류', '상품명', '판매기간', '요약서', '방법서', '약관']
    df = df[columns]
    
    # Excel 파일로 저장
    excel_filename = f'hanwha_life_products_{datetime.now().strftime("%Y%m%d")}.xlsx'
    df.to_excel(excel_filename, index=False)
    print(f"\n총 {len(all_products)}개 상품 정보 수집 완료")
    print(f"파일 저장 완료: {excel_filename}")

except Exception as e:
    print(f"에러 발생: {str(e)}")
    if 'driver' in locals():
        print("현재 페이지 소스:")
        print(driver.page_source[:1000])

finally:
    if 'driver' in locals():
        driver.quit()


판매중지 상품 페이지 접속 완료
데이터 요청 시작
응답 데이터: {'list1': [{'GOODS_TYPE': 'GA', 'GOODS_TYPE_NM': '보장성', 'SELL_TYPE': 'SA', 'SELL_TYPE_NM': '개인'}, {'GOODS_TYPE': 'GB', 'GOODS_TYPE_NM': '연금', 'SELL_TYPE': 'SA', 'SELL_TYPE_NM': '개인'}, {'GOODS_TYPE': 'GC', 'GOODS_TYPE_NM': '생사 혼합', 'SELL_TYPE': 'SA', 'SELL_TYPE_NM': '개인'}, {'GOODS_TYPE': 'GF', 'GOODS_TYPE_NM': '장애인 전용', 'SELL_TYPE': 'SA', 'SELL_TYPE_NM': '개인'}, {'GOODS_TYPE': 'GA', 'GOODS_TYPE_NM': '보장성', 'SELL_TYPE': 'SB', 'SELL_TYPE_NM': '단체'}, {'GOODS_TYPE': 'GB', 'GOODS_TYPE_NM': '연금', 'SELL_TYPE': 'SB', 'SELL_TYPE_NM': '단체'}, {'GOODS_TYPE': 'GC', 'GOODS_TYPE_NM': '생사 혼합', 'SELL_TYPE': 'SB', 'SELL_TYPE_NM': '단체'}, {'GOODS_TYPE': 'GE', 'GOODS_TYPE_NM': '퇴직', 'SELL_TYPE': 'SB', 'SELL_TYPE_NM': '단체'}, {'GOODS_TYPE': 'GF', 'GOODS_TYPE_NM': '장애인 전용', 'SELL_TYPE': 'SB', 'SELL_TYPE_NM': '단체'}, {'GOODS_TYPE': 'GB', 'GOODS_TYPE_NM': '연금', 'SELL_TYPE': 'SC', 'SELL_TYPE_NM': '방카 슈랑스'}, {'GOODS_TYPE': 'GC', 'GOODS_TYPE_NM': '생사 혼합', 'SELL_TYPE': 'SC', 'SELL_

CancelledError: 